In [2]:
# states
# each state transfer to two others: on 0 or on 1
# each transfer has an output

def stringify(a):
    return ''.join([str(i) for i in a])

class FSM_Mealy:
    def __init__(self, data):
        """
        data has many rows each
        (transfer_on_0, out_on_0), (transfer_on_1, output_on_1)
        """
        self.data = data
        
    def run(self, sequence):
        state = 0
        sequence = [int(c) for c in sequence]
        result = []
        
        for c in sequence:
            state, output = self.data[state][c]
            result.append(output)
        
        return stringify(result)
    

class SequenceDetector:
    def __init__(self, targets):
        self.t = targets
        
    def run(self, sequence):
        result = []
        
        for i in range(1, 1+len(sequence)):
            for t in self.t:
                if i < len(t):
                    continue
                
                if sequence[i-len(t):i] == t:
                    result.append(1)
                    break
            else:
                result.append(0)
        
        return stringify(result)
        

expected = SequenceDetector(['1011', '1101'])
# expected = SequenceDetector(['1011'])
        
test = FSM_Mealy([
    ((0,0), (1,0)), # 0: -
    ((2,0), (4,0)), # 1: 1
    ((0,0), (3,0)), # 2: 10
    ((2,0), (4,1)), # 3: 101
    
    ((5,0), (4,0)), # 4: 11
    ((0,0), (3,1)), # 5: 110
])

test_only_last_bit_overlap = FSM_Mealy([
    ((0,0), (1,0)), # 0: -
    ((2,0), (4,0)), # 1: 1
    ((0,0), (3,0)), # 2: 10
    ((2,0), (1,1)), # 3: 101
    
    ((5,0), (4,0)), # 4: 11
    ((0,0), (1,1)), # 5: 110
])



In [3]:
import numpy as np

def generate_input(length):
    return stringify(np.random.randint(0, 2, length))

def check(model1, model2):
    for trial in range(20):
        s = generate_input(50)
        if model1.run(s) != model2.run(s):
            print('error')
            print(s, ' -- input')
            print(model1.run(s), ' -- model1')
            print(model2.run(s), ' -- model2')
            break
    else:
        print('all good!')
        
def check_auto(model1, model2):
    for trial in range(50):
        s = generate_input(20)
        if model1.run(s) != model2.run(s):
            return False
    return True

check(test, expected)

all good!


In [14]:
# four numbers per row, 4 rows
# sixteen numbers

# 8 numbers are links, need to be 0,1,2,3
# 8 numbers are outputs either 0 or 1

from itertools import product

expected = test_only_last_bit_overlap

N = 5
nodes = tuple(range(N)) # = (0,1,2,3)
outs = (0,1)

tries = 0
for n in product(*(N*2)*[nodes]):
    for o in product(*(N*2)*[outs]):
        grid = tuple((
            ((n[i*2], o[i*2]), (n[i*2+1], o[i*2+1]))
            for i in range(N)
        ))
        
        ok = check_auto(FSM_Mealy(grid), expected)
        if ok: 
            print(grid)
        
        tries += 1
        if not tries%16777216:
            print('{:.2f}%'.format(tries/167772.16))

print('100% !!')


100.00%
200.00%
300.00%
400.00%
500.00%
600.00%
700.00%
800.00%
900.00%
1000.00%
1100.00%
1200.00%
1300.00%
1400.00%
1500.00%
1600.00%
1700.00%
1800.00%
1900.00%
2000.00%
2100.00%
2200.00%
2300.00%
2400.00%
2500.00%
2600.00%
2700.00%


KeyboardInterrupt: 

In [5]:
4**8 * 2**8

16777216

In [8]:
(5**10 * 2**10) / (16777216)

596.0464477539062

In [15]:
from itertools import combinations

tuple(combinations('aaabb', 3))

(('a', 'a', 'a'),
 ('a', 'a', 'b'),
 ('a', 'a', 'b'),
 ('a', 'a', 'b'),
 ('a', 'a', 'b'),
 ('a', 'b', 'b'),
 ('a', 'a', 'b'),
 ('a', 'a', 'b'),
 ('a', 'b', 'b'),
 ('a', 'b', 'b'))

In [22]:
def script_for(seq):
    for bit in seq:
        print('; '.join((
            f'force x {bit}',
            'force clk 0',
            'run 50',
            'force clk 1',
            'run 50'
        )))
        
s = '110100110111011'
s = '001101101001011'
script_for(s)

force x 0; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 0; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50
force x 1; force clk 0; run 50; force clk 1; run 50


In [21]:
test_only_last_bit_overlap.run(s)

'000100000100010'